<!-- Styled Box with White Text -->
<div style="background-color: #1B5162; color: #FFFFFF; padding: 15px; border: 1px solid #004085; border-radius: 0px; text-align: center; margin: 10px 0; font-size: 1.5em;">
  <p style="text-align: right; margin: 0px 0; font-size: 1em;"></p>
  <p style="text-align: center; margin: 0px 0; font-size: 1.5em;"><strong>Train machine learning models in SAP Databricks</strong></p>
</div>

## Setting up a Python environment

In [0]:
# ---------------------------------------------------
# Cell : Install Required Python Packages
# ---------------------------------------------------

%pip install faker -qqqq
%restart_python

In [0]:
# ---------------------------------------------------
# Cell : Environment Setup and Library Imports
# ---------------------------------------------------
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from faker import Faker
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import matplotlib.pyplot as plt
import seaborn as sns
import os
import logging

os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"
mlflow.enable_system_metrics_logging()
logging.getLogger("mlflow.system_metrics.system_metrics_monitor").setLevel(logging.ERROR)

# Initialize Faker for English data
fake = Faker('en_US')
np.random.seed(42)
random.seed(42)

print("ℹ️ Library import completed successfully")

<div style="background-color: #d9edf7; color: #31708f; padding: 20px; border-left: 5px solid #31708f; border-radius: 5px; margin: 20px 0;">
  <strong>💡 Tips and Tricks</strong><br/> 
  The Databricks Serverless Notebook environment provides frequently used Python packages pre-installed. If users want to install additional Python packages, they can use the %pip command. <br/>
  
</div>


### Databricks Unity Catalog
<br/>
<div style="background-color: #F9F7F4; padding: 5px; border: 2px solid #1B5162; border-radius: 10px;">
<ul>
  <li><strong style="font-size: 1.2em;">Unity Catalog's 3-level name space</strong>
    <ul>
      <li><span style="font-size: 1.2em;">Catalog: A Catalog in Unity Catalog is a top-level container that organizes schemas for logical data management.</span></li>
      <li><span style="font-size: 1.2em;">Schema: A Schema is a collection within a catalog that groups related tables, views, and data objects.</span></li>
      <li><span style="font-size: 1.2em;">Data Objects: Data objects include tables, views, models, and other entities that store or represent data within a schema.</span></li>
    </ul>
  </li>  
</ul>
</div>

In [0]:
# ---------------------------------------------------
# Cell : Unity Catalog Configuration
# ---------------------------------------------------

# Unity Catalog settings
catalog_name = "workspace"
schema_name = "default"
ml_model_name = "coffee_demand_forecast_model"

# Create catalog and schema using Spark SQL
# spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
# spark.sql(f"USE CATALOG {catalog_name}")
# spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
# spark.sql(f"USE SCHEMA {schema_name}")

print(f"ℹ️ Unity Catalog setup completed: {catalog_name}.{schema_name}")

## Mockup data generation

In [0]:
# ---------------------------------------------------
# Cell : Drop Existing Tables
# ---------------------------------------------------

try:
    spark.sql(f"DROP TABLE IF EXISTS demand_forecast_dataset")
    spark.sql(f"DROP TABLE IF EXISTS kna1")
    spark.sql(f"DROP TABLE IF EXISTS mara")
    spark.sql(f"DROP TABLE IF EXISTS mard")
    spark.sql(f"DROP TABLE IF EXISTS t001w")
    spark.sql(f"DROP TABLE IF EXISTS vbak")
    spark.sql(f"DROP TABLE IF EXISTS vbap")
    print(f"✅ Tables dropped successfully")

except Exception as e:
    print(f"❌ Error occurred while dropping tables: {e}")

In [0]:
# ---------------------------------------------------
# Cell : Generate Regression Data
# ---------------------------------------------------

try:
    # Generate regression features for each store-product combination
    n_stores = 2000
    n_products = 20
    n_samples = n_stores * n_products  # 40,000 samples

    # Generate regression features with controlled noise for 80% accuracy
    X_regression, y_base = make_regression(
        n_samples=n_samples,
        n_features=12,           # Number of features
        n_informative=8,         # Informative features
        noise=20,                # Noise level for ~80% accuracy
        random_state=42
    )

    # Convert features to business-meaningful values
    features_df = pd.DataFrame(X_regression)
    features_df.columns = [
        'store_size_factor',     # Store size factor
        'location_traffic',      # Location traffic
        'season_factor',         # Seasonal factor
        'price_sensitivity',     # Price sensitivity
        'promotion_effect',      # Promotion effect
        'competitor_density',    # Competitor density
        'weather_impact',        # Weather impact
        'weekend_factor',        # Weekend factor
        'trend_factor',          # Trend factor
        'economic_indicator',    # Economic indicator
        'customer_satisfaction', # Customer satisfaction
        'marketing_spend'        # Marketing spend
    ]

    # Normalize and convert to integer (0-100 range)
    for col in features_df.columns:
        # Normalize (0-1)
        normalized = (features_df[col] - features_df[col].min()) / (features_df[col].max() - features_df[col].min())
        # Scale to 0-100 range and convert to integer
        features_df[col] = (normalized * 100).astype(int)

    # Assign stores and products
    features_df['WERKS'] = [f'P{(i//n_products)+1:04d}' for i in range(n_samples)]
    features_df['MATNR'] = [f'C{(i%n_products)+1:05d}' for i in range(n_samples)]

    # Calculate actual demand quantity
    base_demand = 50 + y_base * 10  # Scale base demand
    features_df['actual_demand'] = np.maximum(10, base_demand).astype(int)

    print(f"✅ Regression feature data generated successfully (targeting 80% accuracy): {len(features_df)} records")
    
    # Check data distribution
    print(f"\n📊 Target variable (actual_demand) statistics:")
    print(f"  Mean: {features_df['actual_demand'].mean():.2f}")
    print(f"  Std Dev: {features_df['actual_demand'].std():.2f}")
    print(f"  Min: {features_df['actual_demand'].min()}")
    print(f"  Max: {features_df['actual_demand'].max()}")
    
    display(features_df)

except Exception as e:
    print(f"❌ Error occurred while generating regression feature data: {e}")
    raise

In [0]:
# ---------------------------------------------------
# Cell : Generate SAP Material Master Data - MARA
# ---------------------------------------------------

try:
    # Generate 20 types of coffee beans
    coffee_types = [
        'Colombia_Supremo', 'Ethiopia_Yirgacheffe', 'Brazil_Santos', 'Kenya_AA',
        'Guatemala_Antigua', 'Costa_Rica_Tarrazu', 'Jamaica_Blue_Mountain', 'Hawaii_Kona',
        'Peru_Organic', 'Mexico_Chiapas', 'Indonesia_Sumatra', 'Vietnam_Robusta',
        'India_Malabar', 'Yemen_Mocha', 'Tanzania_Peaberry', 'Nicaragua_Matagalpa',
        'Honduras_Marcala', 'El_Salvador_Pacamara', 'Rwanda_Bourbon', 'Panama_Geisha'
    ]

    # Generate MARA table data
    mara_data = []
    for i, coffee in enumerate(coffee_types, 1):
        mara_data.append({
            'MATNR': f'C{i:05d}',  # Material number
            'MAKTX': coffee,  # Material description
            'MTART': 'ROH',  # Material type (Raw material)
            'MEINS': 'KG',  # Base unit of measure
            'MATKL': '1000',  # Material group
            'BRGEW': round(random.uniform(0.5, 1.0), 2),  # Gross weight
            'NTGEW': round(random.uniform(0.45, 0.95), 2),  # Net weight
            'ERSDA': fake.date_between(start_date='-5y', end_date='-1y'),  # Created on
            'ERNAM': fake.user_name()  # Created by
        })

    df_mara = spark.createDataFrame(pd.DataFrame(mara_data))
    df_mara.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.MARA")

    print(f"✅ MARA table created successfully: {len(mara_data)} materials")
    display(df_mara)

except Exception as e:
    print(f"❌ Error occurred while creating MARA table: {e}")
    raise

In [0]:
# ---------------------------------------------------
# Cell : Generate SAP Plant (Store) Data - T001W
# ---------------------------------------------------

try:
    # Generate 2000 store data
    plant_data = []
    us_states = ['CA', 'NY', 'TX', 'FL', 'IL', 'PA', 'OH', 'GA', 'NC', 'MI', 
                 'NJ', 'VA', 'WA', 'AZ', 'MA', 'TN', 'IN', 'MO', 'MD', 'WI']
    us_cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 
                 'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose',
                 'Austin', 'Jacksonville', 'Fort Worth', 'Columbus', 'Charlotte',
                 'San Francisco', 'Indianapolis', 'Seattle', 'Denver', 'Washington']

    for i in range(1, 2001):
        city = random.choice(us_cities)
        state = random.choice(us_states)
        plant_data.append({
            'WERKS': f'P{i:04d}',  # Plant code
            'NAME1': f'{city} Store #{i%100+1}',  # Plant name
            'LAND1': 'US',  # Country code
            'REGIO': state,  # Region code
            'STRAS': fake.street_address(),  # Street address
            'PSTLZ': fake.zipcode(),  # Postal code
            'ORT01': city,  # City
            'TELF1': fake.phone_number(),  # Telephone
            'EKORG': '1000',  # Purchasing organization
            'VKORG': '1000'  # Sales organization
        })

    df_t001w = spark.createDataFrame(pd.DataFrame(plant_data))
    df_t001w.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.T001W")

    print(f"✅ T001W table created successfully: {len(plant_data)} plants")
    display(df_t001w)

except Exception as e:
    print(f"❌ Error occurred while creating T001W table: {e}")
    raise

In [0]:
# ---------------------------------------------------
# Cell : Generate SAP Sales Order Header - VBAK
# ---------------------------------------------------

try:
    # Generate sales data for the past 6 months
    start_date = datetime.now() - timedelta(days=180)
    vbak_data = []

    for i in range(100000):  # 100,000 sales orders
        order_date = fake.date_between(start_date=start_date, end_date='today')
        vbak_data.append({
            'VBELN': f'S{i+1:010d}',  # Sales document number
            'ERDAT': order_date,  # Created on
            'ERZET': fake.time(),  # Created at
            'ERNAM': fake.user_name(),  # Created by
            'VBTYP': 'C',  # Sales document category (Order)
            'AUART': 'TA',  # Sales document type
            'VKORG': '1000',  # Sales organization
            'VTWEG': '10',  # Distribution channel
            'SPART': '00',  # Division
            'VKGRP': f'{random.randint(1, 10):03d}',  # Sales group
            'VKBUR': f'{random.randint(1, 50):04d}',  # Sales office
            'KUNNR': f'P{random.randint(1, 2000):04d}',  # Customer number (Plant)
            'NETWR': round(random.uniform(100, 5000), 2),  # Net value
            'WAERK': 'USD'  # Currency
        })

    df_vbak = spark.createDataFrame(pd.DataFrame(vbak_data))
    df_vbak.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.VBAK")

    print(f"✅ VBAK table created successfully: {len(vbak_data)} sales orders")
    display(df_vbak)

except Exception as e:
    print(f"❌ Error occurred while creating VBAK table: {e}")
    raise


In [0]:
# ---------------------------------------------------
# Cell : Generate SAP Sales Order Items - VBAP
# ---------------------------------------------------

try:
    # Generate VBAP data using features_df
    vbap_data = []
    vbeln_counter = 1

    for idx, row in features_df.iterrows():
        # Generate multiple orders for each store-product combination
        n_orders = random.randint(5, 20)

        for _ in range(n_orders):
            # Generate order quantity based on actual demand with variation
            base_qty = row['actual_demand']
            variation_factor = random.uniform(0.7, 1.3)
            order_qty = max(1, int(base_qty * variation_factor))

            vbap_data.append({
                'VBELN': f'S{vbeln_counter:010d}',  # Sales document number
                'POSNR': '000010',  # Item number
                'MATNR': row['MATNR'],  # Material number
                'MATKL': '1000',  # Material group
                'WERKS': row['WERKS'],  # Plant
                'LGORT': '1000',  # Storage location
                'KWMENG': float(order_qty),  # Order quantity
                'VRKME': 'KG',  # Sales unit
                'NETPR': round(random.uniform(15, 30), 2),  # Net price
                'NETWR': round(order_qty * random.uniform(15, 30), 2),  # Net value
                'WAERK': 'USD',  # Currency
                'VSTEL': '1000',  # Shipping point
                'ROUTE': f'R{random.randint(1, 100):03d}'  # Route
            })
            vbeln_counter += 1

    # Limit to 200,000 records
    vbap_data = vbap_data[:200000]

    df_vbap = spark.createDataFrame(pd.DataFrame(vbap_data))
    df_vbap.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.VBAP")

    print(f"✅ VBAP table created successfully: {len(vbap_data)} sales items")
    display(df_vbap)

except Exception as e:
    print(f"❌ Error occurred while creating VBAP table: {e}")
    raise


In [0]:
# ---------------------------------------------------
# Cell : Generate SAP Inventory Data - MARD
# ---------------------------------------------------

try:
    # Generate inventory data for each store-product combination
    mard_data = []

    for werks in [f'P{i:04d}' for i in range(1, 2001)]:
        for matnr in [f'C{i:05d}' for i in range(1, 21)]:
            # Get demand information from features_df
            demand_info = features_df[(features_df['WERKS'] == werks) & 
                                    (features_df['MATNR'] == matnr)]
            
            if not demand_info.empty:
                avg_demand = demand_info['actual_demand'].values[0]
                # Current stock with variation
                current_stock = max(0, int(avg_demand * random.uniform(0.5, 2.0)))
                safety_stock = max(10, int(avg_demand * random.uniform(0.3, 0.7)))
            else:
                current_stock = random.randint(0, 200)
                safety_stock = random.randint(10, 50)
                
            mard_data.append({
                'MATNR': matnr,  # Material number
                'WERKS': werks,  # Plant
                'LGORT': '1000',  # Storage location
                'LABST': float(current_stock),  # Available stock
                'UMLME': float(random.randint(0, 20)),  # Stock in transfer
                'INSME': float(random.randint(0, 10)),  # Stock in quality inspection
                'SPEME': float(random.randint(0, 5)),  # Blocked stock
                'EISBE': float(safety_stock),  # Safety stock
                'LGPBE': f'L{random.randint(1, 100):03d}'  # Storage bin
            })

    df_mard = spark.createDataFrame(pd.DataFrame(mard_data))
    df_mard.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.MARD")

    print(f"✅ MARD table created successfully: {len(mard_data)} inventory records")
    display(df_mard)

except Exception as e:
    print(f"❌ Error occurred while creating MARD table: {e}")
    raise

## Data manipulation for data marts for ML analysis

### List of SAP tables used in this demo
<br/>
<div style="background-color: #F9F7F4; padding: 5px; border: 2px solid #1B5162; border-radius: 10px;">
<ul>
  <li><strong style="font-size: 1.2em;">MARA – Material Master </strong>
    <ul>
      <li><span style="font-size: 1.2em;">MARA contains core material master data used across SAP modules, including material number, description, type, base unit of measure, material group, gross and net weight, and creation details. It serves as the central reference for all processes involving materials, ensuring consistency in purchasing, sales, production, and inventory management.</span></li>
    </ul>
  </li>  
</ul>
<ul>
  <li><strong style="font-size: 1.2em;">T001W – Plant Master</strong>
    <ul>
      <li><span style="font-size: 1.2em;">T001W contains master data for plants, covering plant codes, names, addresses, country and region codes, telephone numbers, and associated purchasing and sales organizations. It defines key organizational units used in logistics, production, sales, and financial processes across the enterprise.</span></li>
    </ul>
  </li>  
</ul>
<ul>
  <li><strong style="font-size: 1.2em;">VBAK – Sales Order Header</strong>
    <ul>
      <li><span style="font-size: 1.2em;">VBAK contains header-level information for sales orders, such as sales document number, creation date/time, document category and type, sales organization, distribution channel, division, customer number, and net order value with currency. It stores data common to all items within an order and supports the overall sales process flow.</span></li>
    </ul>
  </li>  
</ul>
<ul>
  <li><strong style="font-size: 1.2em;">VBAP – Sales Order Items</strong>
    <ul>
      <li><span style="font-size: 1.2em;">VBAP contains item-level details for sales orders, linked to VBAK headers, including material numbers, material groups, plant and storage location assignments, ordered quantity, sales unit, net price, net value, currency, shipping point, and route. Each record represents a specific product or service sold and drives delivery, billing, and revenue recognition activities.</span></li>
    </ul>
  </li>  
</ul>
<ul>
  <li><strong style="font-size: 1.2em;">MARD – Storage Location Stock</strong>
    <ul>
      <li><span style="font-size: 1.2em;">MARD contains stock information for materials at the plant and storage location level, tracking available stock, stock in transfer, stock under quality inspection, blocked stock, safety stock, and storage bin details. It supports warehouse operations, stock valuation, and material availability checks for procurement and sales.</span></li>
    </ul>
  </li>  
</ul>
</div>

In [0]:
# ---------------------------------------------------
# Cell : MARA Table
# ---------------------------------------------------

spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.MARA").display()


In [0]:
# ---------------------------------------------------
# Cell : T001W Table
# ---------------------------------------------------

spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.T001W").display()


In [0]:
# ---------------------------------------------------
# Cell : VBAK Table
# ---------------------------------------------------

spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.VBAK").display()


In [0]:
# ---------------------------------------------------
# Cell : VBAP Table
# ---------------------------------------------------

spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.VBAP").display()


In [0]:
# ---------------------------------------------------
# Cell : MARD Table
# ---------------------------------------------------

spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.MARD").display()


In [0]:
# ---------------------------------------------------
# Cell : Create Final Demand Forecast Dataset
# ---------------------------------------------------

try:
    # Join all data to create final dataset
    query_final = f"""
    WITH base_data AS (
        SELECT 
            s.WERKS,
            s.MATNR,
            s.avg_sales_qty,
            s.total_sales_qty,
            s.order_count,
            s.avg_price,
            s.min_qty,
            s.max_qty,
            s.std_qty,
            s.coefficient_variation,
            i.LABST as current_stock,
            i.EISBE as safety_stock,
            (i.LABST + i.UMLME - i.INSME) as total_available,
            st.city,
            st.region,
            st.city_tier,
            p.product_name,
            p.gross_weight,
            p.product_grade
        FROM (
            SELECT WERKS, MATNR,
                AVG(KWMENG) as avg_sales_qty,
                SUM(KWMENG) as total_sales_qty,
                COUNT(*) as order_count,
                AVG(NETPR) as avg_price,
                MIN(KWMENG) as min_qty,
                MAX(KWMENG) as max_qty,
                STDDEV(KWMENG) as std_qty,
                CASE WHEN AVG(KWMENG) > 0 THEN STDDEV(KWMENG) / AVG(KWMENG) ELSE 0 END as coefficient_variation
            FROM {catalog_name}.{schema_name}.VBAP
            GROUP BY WERKS, MATNR
        ) s
        LEFT JOIN {catalog_name}.{schema_name}.MARD i 
            ON s.WERKS = i.WERKS AND s.MATNR = i.MATNR
        LEFT JOIN (
            SELECT WERKS, ORT01 as city, REGIO as region,
                CASE 
                    WHEN ORT01 IN ('New York', 'Los Angeles', 'Chicago') THEN 3
                    WHEN ORT01 IN ('Houston', 'Phoenix', 'Philadelphia') THEN 2
                    ELSE 1
                END as city_tier
            FROM {catalog_name}.{schema_name}.T001W
        ) st ON s.WERKS = st.WERKS
        LEFT JOIN (
            SELECT MATNR, MAKTX as product_name, BRGEW as gross_weight,
                CASE 
                    WHEN MAKTX LIKE '%Blue_Mountain%' OR MAKTX LIKE '%Kona%' OR MAKTX LIKE '%Geisha%' THEN 3
                    WHEN MAKTX LIKE '%Yirgacheffe%' OR MAKTX LIKE '%AA%' THEN 2
                    ELSE 1
                END as product_grade
            FROM {catalog_name}.{schema_name}.MARA
        ) p ON s.MATNR = p.MATNR
    )
    SELECT DISTINCT
        WERKS,
        MATNR,
        product_name,
        city,
        region,
        city_tier,
        product_grade,
        avg_sales_qty,
        total_sales_qty,
        order_count,
        avg_price,
        min_qty,
        max_qty,
        COALESCE(std_qty, 0) as std_qty,
        COALESCE(coefficient_variation, 0) as coefficient_variation,
        COALESCE(current_stock, 0) as current_stock,
        COALESCE(safety_stock, 0) as safety_stock,
        COALESCE(total_available, 0) as total_available,
        gross_weight,
        -- Derived features
        CASE WHEN current_stock > 0 THEN avg_sales_qty / current_stock ELSE 0 END as stock_turnover,
        current_stock - safety_stock as available_stock,
        CASE WHEN avg_sales_qty > 0 THEN current_stock / avg_sales_qty ELSE 0 END as days_of_stock,
        avg_price / 10 as price_tier,
        order_count / 10 as order_frequency_score,
        -- Target variable: next order quantity
        GREATEST(0, 
            avg_sales_qty * 1.2 + 
            safety_stock * 0.5 - 
            current_stock * 0.8
        ) as next_order_qty
    FROM base_data
    WHERE avg_sales_qty IS NOT NULL
    """

    df_final = spark.sql(query_final)
    df_final.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.demand_forecast_dataset")

    print(f"✅ Final demand forecast dataset created successfully: {df_final.count()} records")
    display(df_final)

except Exception as e:
    print(f"❌ Error occurred while creating final demand forecast dataset: {e}")
    raise

### Variables for machine learning models
<br/>
<div style="background-color: #F9F7F4; padding: 5px; border: 2px solid #1B5162; border-radius: 10px;">
<ul>
  <li><strong style="font-size: 1.2em;">Target Variable</strong>
    <ul>      
      <li><span style="font-size: 1.2em;">next_order_qty : The estimated quantity required for the next order, taking into consideration the average sales volume in the past, safety stock, and current inventory.</span></li>
    </ul>
  </li>  
  <li><strong style="font-size: 1.2em;">Feature Variables</strong>
    <ul>      
      <li><span style="font-size: 1.2em;">The feature variables consist of plant and product identification information (factory code, material name, city/region, grade), sales indicators (average and total sales volume, number of orders, price, volatility), inventory indicators (current and safety stock, available stock, weight), and derived indicators (inventory turnover rate, inventory days, price and order frequency score).</span></li>
    </ul>
  </li>  
</ul>
</div>

## Exploratory Data Analysis

In [0]:
# ---------------------------------------------------
# Cell : MLflow Setup and Experiment Initialization
# ---------------------------------------------------

# Unity Catalog and MLflow integration setup
mlflow.set_registry_uri("databricks-uc")

# Get current user information
current_user = spark.sql("SELECT current_user() AS user").collect()[0]['user']
experiment_name = f"/Users/{current_user}/coffee_demand_forecast"
mlflow.set_experiment(experiment_name)

# Set model registry name
model_name = f"{catalog_name}.{schema_name}.{ml_model_name}"

print(f"ℹ️ MLflow experiment setup: {experiment_name}")
print(f"ℹ️ Model registry: {model_name}")

In [0]:
# ---------------------------------------------------
# Cell : Exploratory Data Analysis (EDA) - Basic Statistics and Distributions
# ---------------------------------------------------

# Load the final dataset for EDA
df_eda = spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.demand_forecast_dataset")
pdf_eda = df_eda.toPandas()

print(f"📊 Dataset Overview:")
print(f"  - Total Records: {len(pdf_eda):,}")
print(f"  - Total Features: {len(pdf_eda.columns)}")
print(f"  - Memory Usage: {pdf_eda.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Basic statistical summary
print(f"\n📈 Statistical Summary:")
print(pdf_eda.describe())

# Data distribution visualization
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
fig.suptitle('Distribution of Key Variables', fontsize=16, fontweight='bold')

# Target variable distribution
axes[0, 0].hist(pdf_eda['next_order_qty'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Next Order Quantity Distribution')
axes[0, 0].set_xlabel('Order Quantity (KG)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

# Average sales quantity distribution
axes[0, 1].hist(pdf_eda['avg_sales_qty'], bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0, 1].set_title('Average Sales Quantity Distribution')
axes[0, 1].set_xlabel('Average Sales (KG)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3)

# Current stock distribution
axes[0, 2].hist(pdf_eda['current_stock'], bins=50, alpha=0.7, color='orange', edgecolor='black')
axes[0, 2].set_title('Current Stock Distribution')
axes[0, 2].set_xlabel('Current Stock (KG)')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].grid(True, alpha=0.3)

# Order count distribution
axes[1, 0].hist(pdf_eda['order_count'], bins=30, alpha=0.7, color='pink', edgecolor='black')
axes[1, 0].set_title('Order Count Distribution')
axes[1, 0].set_xlabel('Number of Orders')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3)

# Average price distribution
axes[1, 1].hist(pdf_eda['avg_price'], bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
axes[1, 1].set_title('Average Price Distribution')
axes[1, 1].set_xlabel('Average Price (USD)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].grid(True, alpha=0.3)

# Safety stock distribution
axes[1, 2].hist(pdf_eda['safety_stock'], bins=50, alpha=0.7, color='gold', edgecolor='black')
axes[1, 2].set_title('Safety Stock Distribution')
axes[1, 2].set_xlabel('Safety Stock (KG)')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].grid(True, alpha=0.3)

# City tier distribution
city_tier_counts = pdf_eda['city_tier'].value_counts().sort_index()
axes[2, 0].bar(city_tier_counts.index, city_tier_counts.values, alpha=0.7, color='purple', edgecolor='black')
axes[2, 0].set_title('City Tier Distribution')
axes[2, 0].set_xlabel('City Tier')
axes[2, 0].set_ylabel('Count')
axes[2, 0].grid(True, alpha=0.3)

# Product grade distribution
product_grade_counts = pdf_eda['product_grade'].value_counts().sort_index()
axes[2, 1].bar(product_grade_counts.index, product_grade_counts.values, alpha=0.7, color='teal', edgecolor='black')
axes[2, 1].set_title('Product Grade Distribution')
axes[2, 1].set_xlabel('Product Grade')
axes[2, 1].set_ylabel('Count')
axes[2, 1].grid(True, alpha=0.3)

# Stock turnover distribution
axes[2, 2].hist(pdf_eda['stock_turnover'], bins=50, alpha=0.7, color='navy', edgecolor='black')
axes[2, 2].set_title('Stock Turnover Distribution')
axes[2, 2].set_xlabel('Stock Turnover Ratio')
axes[2, 2].set_ylabel('Frequency')
axes[2, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Basic distribution analysis completed")

In [0]:
# ---------------------------------------------------
# Cell : EDA - Correlation Analysis and Heatmap
# ---------------------------------------------------

# Select numerical columns for correlation analysis
numerical_cols = [
    'city_tier', 'product_grade', 'avg_sales_qty', 'order_count',
    'avg_price', 'min_qty', 'max_qty', 'std_qty', 'coefficient_variation',
    'current_stock', 'safety_stock', 'total_available', 'gross_weight',
    'stock_turnover', 'available_stock', 'days_of_stock', 'price_tier',
    'order_frequency_score', 'next_order_qty'
]

# Calculate correlation matrix
correlation_matrix = pdf_eda[numerical_cols].corr()

# Create correlation heatmap
plt.figure(figsize=(16, 14))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
heatmap = sns.heatmap(correlation_matrix, 
                      mask=mask,
                      annot=True, 
                      cmap='RdYlBu_r', 
                      center=0,
                      square=True,
                      fmt='.2f',
                      cbar_kws={'shrink': 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Identify highly correlated features
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.7:  # High correlation threshold
            high_corr_pairs.append((
                correlation_matrix.columns[i], 
                correlation_matrix.columns[j], 
                corr_value
            ))

print(f"\n🔍 Highly Correlated Feature Pairs (|correlation| > 0.7):")
for feat1, feat2, corr in high_corr_pairs:
    print(f"  - {feat1} ↔ {feat2}: {corr:.3f}")

# Target variable correlations
target_correlations = correlation_matrix['next_order_qty'].abs().sort_values(ascending=False)
print(f"\n🎯 Features Most Correlated with Target Variable:")
for feature, corr in target_correlations.head(10).items():
    if feature != 'next_order_qty':
        print(f"  - {feature}: {corr:.3f}")

print("✅ Correlation analysis completed")

In [0]:
# ---------------------------------------------------
# Cell : EDA - Store and Product Analysis
# ---------------------------------------------------

# Store performance analysis
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Store and Product Performance Analysis', fontsize=16, fontweight='bold')

# Top 20 stores by total sales
store_sales = pdf_eda.groupby('WERKS')['total_sales_qty'].sum().sort_values(ascending=False).head(20)
axes[0, 0].bar(range(len(store_sales)), store_sales.values, alpha=0.7, color='steelblue')
axes[0, 0].set_title('Top 20 Stores by Total Sales')
axes[0, 0].set_xlabel('Store Rank')
axes[0, 0].set_ylabel('Total Sales Quantity (KG)')
axes[0, 0].grid(True, alpha=0.3)

# Product performance analysis
product_sales = pdf_eda.groupby('product_name')['total_sales_qty'].sum().sort_values(ascending=False)
axes[0, 1].bar(range(len(product_sales)), product_sales.values, alpha=0.7, color='forestgreen')
axes[0, 1].set_title('Product Sales Performance')
axes[0, 1].set_xlabel('Product Rank')
axes[0, 1].set_ylabel('Total Sales Quantity (KG)')
axes[0, 1].grid(True, alpha=0.3)

# Average order quantity by product grade
avg_order_by_grade = pdf_eda.groupby('product_grade')['next_order_qty'].mean()
axes[0, 2].bar(avg_order_by_grade.index, avg_order_by_grade.values, alpha=0.7, color='orange')
axes[0, 2].set_title('Average Order Quantity by Product Grade')
axes[0, 2].set_xlabel('Product Grade')
axes[0, 2].set_ylabel('Average Order Quantity (KG)')
axes[0, 2].grid(True, alpha=0.3)

# City tier analysis
city_sales = pdf_eda.groupby('city_tier')['avg_sales_qty'].mean()
axes[1, 0].bar(city_sales.index, city_sales.values, alpha=0.7, color='purple')
axes[1, 0].set_title('Average Sales by City Tier')
axes[1, 0].set_xlabel('City Tier')
axes[1, 0].set_ylabel('Average Sales Quantity (KG)')
axes[1, 0].grid(True, alpha=0.3)

# Regional analysis (top 10 regions)
region_sales = pdf_eda.groupby('region')['total_sales_qty'].sum().sort_values(ascending=False).head(10)
axes[1, 1].bar(range(len(region_sales)), region_sales.values, alpha=0.7, color='red')
axes[1, 1].set_title('Top 10 Regions by Sales')
axes[1, 1].set_xlabel('Region Rank')
axes[1, 1].set_ylabel('Total Sales Quantity (KG)')
axes[1, 1].set_xticks(range(len(region_sales)))
axes[1, 1].set_xticklabels(region_sales.index, rotation=45, ha='right')
axes[1, 1].grid(True, alpha=0.3)

# Price vs Sales relationship
axes[1, 2].scatter(pdf_eda['avg_price'], pdf_eda['avg_sales_qty'], alpha=0.6, s=20, color='darkblue')
axes[1, 2].set_title('Price vs Sales Relationship')
axes[1, 2].set_xlabel('Average Price (USD)')
axes[1, 2].set_ylabel('Average Sales Quantity (KG)')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Detailed product analysis
print(f"\n📊 Product Performance Summary:")
product_summary = pdf_eda.groupby('product_name').agg({
    'avg_sales_qty': 'mean',
    'avg_price': 'mean',
    'current_stock': 'mean',
    'next_order_qty': 'mean'
}).round(2).sort_values('avg_sales_qty', ascending=False)

print(product_summary.head(10))
print("✅ Store and product analysis completed")

In [0]:
# ---------------------------------------------------
# Cell : EDA - Inventory and Stock Analysis
# ---------------------------------------------------

# Inventory analysis
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Inventory and Stock Analysis', fontsize=16, fontweight='bold')

# Stock level distribution by product grade
product_grades = sorted(pdf_eda['product_grade'].unique())
stock_data = [pdf_eda[pdf_eda['product_grade'] == grade]['current_stock'] for grade in product_grades]
axes[0, 0].boxplot(stock_data, labels=product_grades)
axes[0, 0].set_title('Stock Levels by Product Grade')
axes[0, 0].set_xlabel('Product Grade')
axes[0, 0].set_ylabel('Current Stock (KG)')
axes[0, 0].grid(True, alpha=0.3)

# Safety stock vs current stock
axes[0, 1].scatter(pdf_eda['safety_stock'], pdf_eda['current_stock'], alpha=0.6, s=20, color='green')
axes[0, 1].plot([0, pdf_eda['safety_stock'].max()], [0, pdf_eda['safety_stock'].max()], 'r--', label='Safety = Current')
axes[0, 1].set_title('Safety Stock vs Current Stock')
axes[0, 1].set_xlabel('Safety Stock (KG)')
axes[0, 1].set_ylabel('Current Stock (KG)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Stock shortage analysis
pdf_eda['stock_shortage'] = pdf_eda['current_stock'] < pdf_eda['safety_stock']
shortage_by_grade = pdf_eda.groupby('product_grade')['stock_shortage'].mean() * 100
axes[0, 2].bar(shortage_by_grade.index, shortage_by_grade.values, alpha=0.7, color='red')
axes[0, 2].set_title('Stock Shortage Rate by Product Grade (%)')
axes[0, 2].set_xlabel('Product Grade')
axes[0, 2].set_ylabel('Shortage Rate (%)')
axes[0, 2].grid(True, alpha=0.3)

# Days of stock distribution
axes[1, 0].hist(pdf_eda['days_of_stock'], bins=50, alpha=0.7, color='brown', edgecolor='black')
axes[1, 0].set_title('Days of Stock Distribution')
axes[1, 0].set_xlabel('Days of Stock')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3)

# Stock turnover by city tier
turnover_by_city = pdf_eda.groupby('city_tier')['stock_turnover'].mean()
axes[1, 1].bar(turnover_by_city.index, turnover_by_city.values, alpha=0.7, color='teal')
axes[1, 1].set_title('Average Stock Turnover by City Tier')
axes[1, 1].set_xlabel('City Tier')
axes[1, 1].set_ylabel('Stock Turnover Ratio')
axes[1, 1].grid(True, alpha=0.3)

# Available stock analysis
axes[1, 2].scatter(pdf_eda['available_stock'], pdf_eda['next_order_qty'], alpha=0.6, s=20, color='purple')
axes[1, 2].set_title('Available Stock vs Next Order Quantity')
axes[1, 2].set_xlabel('Available Stock (KG)')
axes[1, 2].set_ylabel('Next Order Quantity (KG)')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Stock shortage statistics
shortage_stats = pdf_eda.groupby(['city_tier', 'product_grade'])['stock_shortage'].agg(['count', 'sum', 'mean']).round(3)
shortage_stats.columns = ['Total_Stores', 'Shortage_Count', 'Shortage_Rate']
shortage_stats['Shortage_Rate'] = shortage_stats['Shortage_Rate'] * 100

print(f"\n🚨 Stock Shortage Analysis by City Tier and Product Grade:")
print(shortage_stats)

print("✅ Inventory and stock analysis completed")

In [0]:
# ---------------------------------------------------
# Cell : EDA - Advanced Scatter Plot Matrix and Outlier Analysis
# ---------------------------------------------------

# Select key features for scatter plot matrix
key_features = [
    'avg_sales_qty', 'current_stock', 'avg_price', 'order_count',
    'stock_turnover', 'days_of_stock', 'next_order_qty'
]

# Create scatter plot matrix
fig, axes = plt.subplots(len(key_features), len(key_features), figsize=(20, 20))
fig.suptitle('Scatter Plot Matrix of Key Features', fontsize=16, fontweight='bold')

for i in range(len(key_features)):
    for j in range(len(key_features)):
        if i == j:
            # Diagonal: histograms
            axes[i, j].hist(pdf_eda[key_features[i]], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
            axes[i, j].set_title(f'{key_features[i]}')
        else:
            # Off-diagonal: scatter plots
            sample_size = min(2000, len(pdf_eda))  # Sample for performance
            sample_data = pdf_eda.sample(n=sample_size, random_state=42)
            axes[i, j].scatter(sample_data[key_features[j]], sample_data[key_features[i]], 
                             alpha=0.5, s=10, color='darkblue')
            
        if i == len(key_features) - 1:
            axes[i, j].set_xlabel(key_features[j], rotation=45, ha='right')
        if j == 0:
            axes[i, j].set_ylabel(key_features[i])
        
        axes[i, j].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Outlier detection using IQR method
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
fig.suptitle('Outlier Detection Analysis (Box Plots)', fontsize=16, fontweight='bold')

outlier_features = ['next_order_qty', 'avg_sales_qty', 'current_stock', 'avg_price',
                   'order_count', 'stock_turnover', 'days_of_stock', 'coefficient_variation']

for idx, feature in enumerate(outlier_features):
    row = idx // 4
    col = idx % 4
    
    # Calculate outliers using IQR
    Q1 = pdf_eda[feature].quantile(0.25)
    Q3 = pdf_eda[feature].quantile(0.75)
    IQR = Q3 - Q1
    outlier_threshold_low = Q1 - 1.5 * IQR
    outlier_threshold_high = Q3 + 1.5 * IQR
    
    outliers = pdf_eda[(pdf_eda[feature] < outlier_threshold_low) | 
                       (pdf_eda[feature] > outlier_threshold_high)]
    
    # Box plot
    box_plot = axes[row, col].boxplot(pdf_eda[feature], patch_artist=True)
    box_plot['boxes'][0].set_facecolor('lightblue')
    axes[row, col].set_title(f'{feature}\n({len(outliers)} outliers)')
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Outlier summary
print(f"\n🔍 Outlier Detection Summary (using IQR method):")
for feature in outlier_features:
    Q1 = pdf_eda[feature].quantile(0.25)
    Q3 = pdf_eda[feature].quantile(0.75)
    IQR = Q3 - Q1
    outlier_threshold_low = Q1 - 1.5 * IQR
    outlier_threshold_high = Q3 + 1.5 * IQR
    
    outliers = pdf_eda[(pdf_eda[feature] < outlier_threshold_low) | 
                       (pdf_eda[feature] > outlier_threshold_high)]
    
    outlier_percentage = (len(outliers) / len(pdf_eda)) * 100
    print(f"  - {feature}: {len(outliers)} outliers ({outlier_percentage:.2f}%)")

print("✅ Advanced analysis and outlier detection completed")

In [0]:
# ---------------------------------------------------
# Cell : EDA - Business Insights and Pattern Analysis
# ---------------------------------------------------

# Business insights visualization
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Business Insights and Pattern Analysis', fontsize=16, fontweight='bold')

# Order frequency vs Average order size
pdf_eda['order_size_category'] = pd.cut(pdf_eda['avg_sales_qty'], 
                                       bins=[0, 50, 100, 150, float('inf')], 
                                       labels=['Small', 'Medium', 'Large', 'XLarge'])

order_freq_analysis = pdf_eda.groupby('order_size_category')['order_count'].mean()
axes[0, 0].bar(order_freq_analysis.index, order_freq_analysis.values, alpha=0.7, color='coral')
axes[0, 0].set_title('Order Frequency by Order Size Category')
axes[0, 0].set_xlabel('Order Size Category')
axes[0, 0].set_ylabel('Average Order Count')
axes[0, 0].grid(True, alpha=0.3)

# Price sensitivity analysis
price_segments = pd.cut(pdf_eda['avg_price'], bins=5, labels=['Low', 'Med-Low', 'Medium', 'Med-High', 'High'])
price_demand = pdf_eda.groupby(price_segments)['avg_sales_qty'].mean()
axes[0, 1].bar(price_demand.index, price_demand.values, alpha=0.7, color='gold')
axes[0, 1].set_title('Average Demand by Price Segment')
axes[0, 1].set_xlabel('Price Segment')
axes[0, 1].set_ylabel('Average Sales Quantity (KG)')
axes[0, 1].grid(True, alpha=0.3)

# Seasonality proxy (using order variability)
cv_segments = pd.cut(pdf_eda['coefficient_variation'], 
                    bins=[0, 0.3, 0.6, 1.0, float('inf')], 
                    labels=['Stable', 'Moderate', 'Variable', 'Highly Variable'])
seasonal_pattern = pdf_eda.groupby(cv_segments)['next_order_qty'].mean()
axes[0, 2].bar(seasonal_pattern.index, seasonal_pattern.values, alpha=0.7, color='lightgreen')
axes[0, 2].set_title('Order Quantity by Demand Variability')
axes[0, 2].set_xlabel('Demand Variability')
axes[0, 2].set_ylabel('Average Next Order Quantity (KG)')
axes[0, 2].grid(True, alpha=0.3)

# Geographic performance heatmap
city_performance = pdf_eda.groupby(['region', 'city_tier']).agg({
    'avg_sales_qty': 'mean',
    'next_order_qty': 'mean'
}).reset_index()

pivot_sales = city_performance.pivot(index='region', columns='city_tier', values='avg_sales_qty')
im1 = axes[1, 0].imshow(pivot_sales.values, cmap='YlOrRd', aspect='auto')
axes[1, 0].set_title('Average Sales by Region and City Tier')
axes[1, 0].set_xlabel('City Tier')
axes[1, 0].set_ylabel('Region')
axes[1, 0].set_xticks(range(len(pivot_sales.columns)))
axes[1, 0].set_xticklabels(pivot_sales.columns)
axes[1, 0].set_yticks(range(len(pivot_sales.index)))
axes[1, 0].set_yticklabels(pivot_sales.index)
plt.colorbar(im1, ax=axes[1, 0], shrink=0.8)

# Inventory efficiency analysis
pdf_eda['inventory_efficiency'] = pdf_eda['avg_sales_qty'] / (pdf_eda['current_stock'] + 1)
efficiency_by_grade = pdf_eda.groupby('product_grade')['inventory_efficiency'].mean()
axes[1, 1].bar(efficiency_by_grade.index, efficiency_by_grade.values, alpha=0.7, color='navy')
axes[1, 1].set_title('Inventory Efficiency by Product Grade')
axes[1, 1].set_xlabel('Product Grade')
axes[1, 1].set_ylabel('Inventory Efficiency Ratio')
axes[1, 1].grid(True, alpha=0.3)

# Demand prediction complexity analysis
pdf_eda['prediction_complexity'] = pdf_eda['std_qty'] / (pdf_eda['avg_sales_qty'] + 1)
complexity_dist = pdf_eda['prediction_complexity']
axes[1, 2].hist(complexity_dist, bins=50, alpha=0.7, color='darkred', edgecolor='black')
axes[1, 2].set_title('Demand Prediction Complexity Distribution')
axes[1, 2].set_xlabel('Prediction Complexity Score')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Generate business insights report
print(f"\n💡 Key Business Insights:")

# Top performing products
top_products = pdf_eda.groupby('product_name')['avg_sales_qty'].mean().sort_values(ascending=False).head(5)
print(f"\n🏆 Top 5 Performing Products:")
for product, sales in top_products.items():
    print(f"  - {product}: {sales:.2f} KG average sales")

# Most challenging products to predict
challenging_products = pdf_eda.groupby('product_name')['coefficient_variation'].mean().sort_values(ascending=False).head(5)
print(f"\n🎯 Most Challenging Products to Predict:")
for product, cv in challenging_products.items():
    print(f"  - {product}: {cv:.3f} coefficient of variation")

# Regional insights
regional_performance = pdf_eda.groupby('region').agg({
    'avg_sales_qty': 'mean',
    'avg_price': 'mean',
    'stock_shortage': 'mean'
}).round(3)
regional_performance['stock_shortage'] = regional_performance['stock_shortage'] * 100

print(f"\n🌍 Regional Performance Summary:")
print(regional_performance.head())

print("✅ Business insights and pattern analysis completed")

In [0]:
# ---------------------------------------------------
# Cell : Data Loading and Preprocessing
# ---------------------------------------------------

# Load data
df_ml = spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.demand_forecast_dataset")
pdf_ml = df_ml.toPandas()

# Select numerical features
feature_columns = [
    'city_tier', 'product_grade', 'avg_sales_qty', 'order_count',
    'avg_price', 'min_qty', 'max_qty', 'std_qty', 'coefficient_variation',
    'current_stock', 'safety_stock', 'total_available', 'gross_weight', 
    'stock_turnover', 'available_stock', 'days_of_stock', 'price_tier',
    'order_frequency_score'
]

# Target variable
target_column = 'next_order_qty'

# Handle missing values
pdf_ml[feature_columns] = pdf_ml[feature_columns].fillna(0)
pdf_ml[target_column] = pdf_ml[target_column].fillna(0)

# Handle infinite values
pdf_ml = pdf_ml.replace([np.inf, -np.inf], 0)

# Separate features and target
X = pdf_ml[feature_columns]
y = pdf_ml[target_column]

print(f"ℹ️ Number of features: {len(feature_columns)}")
print(f"ℹ️ Total samples: {len(X)}")
print(f"ℹ️ Target variable mean: {y.mean():.2f}")
print(f"ℹ️ Target variable std: {y.std():.2f}")

# Check data distribution
print(f"\n📊 Feature statistics:")
print(X.describe())

## Machine Learning model training with MLflow

### MLflow
<br/>
<div style="background-color: #F9F7F4; padding: 5px; border: 2px solid #1B5162; border-radius: 10px;">
<ul>
  <li><strong style="font-size: 1.2em;">What is MLflow?</strong>
    <ul>      
      <li><span style="font-size: 1.2em;">MLflow is an open source platform for developing models and generative AI applications. It has the following primary components.</span></li>
    </ul>
  </li>  
  <li><strong style="font-size: 1.2em;">MLflow components</strong>
    <ul>      
      <li><span style="font-size: 1.2em;">Tracking : Allows you to track experiments to record and compare parameters and results.</span></li>
      <li><span style="font-size: 1.2em;">Models : Allow you to manage and deploy models from various ML libraries to various model serving and inference platforms.</span></li>
      <li><span style="font-size: 1.2em;">Model Registry : Allows you to manage the model deployment process from staging to production, with model versioning and annotation capabilities.</span></li>
      <li><span style="font-size: 1.2em;">AI agent evaluation and tracing : Allows you to develop high-quality AI agents by helping you compare, evaluate, and troubleshoot agents.</span></li>
      <li><span style="font-size: 1.2em;">Supported Language : MLflow supports Java, Python, R, and REST APIs.</span></li>
    </ul>
  </li>  
</ul>
</div>

In [0]:
# ---------------------------------------------------
# Cell : Data Splitting and Scaling
# ---------------------------------------------------

# Train/test data split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"ℹ️ Training data: {X_train.shape}")
print(f"ℹ️ Test data: {X_test.shape}")

In [0]:
# ---------------------------------------------------
# Cell : Linear Regression Model Training
# ---------------------------------------------------

# Start MLflow run - Linear Regression
with mlflow.start_run(run_name="Linear_Regression") as run:

    # Train model
    lr_model = LinearRegression()
    lr_model.fit(X_train_scaled, y_train)

    # Predictions
    y_pred_train_lr = lr_model.predict(X_train_scaled)
    y_pred_test_lr = lr_model.predict(X_test_scaled)

    # Performance evaluation
    train_rmse_lr = np.sqrt(mean_squared_error(y_train, y_pred_train_lr))
    test_rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_test_lr))
    train_r2_lr = r2_score(y_train, y_pred_train_lr)
    test_r2_lr = r2_score(y_test, y_pred_test_lr)
    train_mae_lr = mean_absolute_error(y_train, y_pred_train_lr)
    test_mae_lr = mean_absolute_error(y_test, y_pred_test_lr)

    # Log metrics
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("n_features", len(feature_columns))
    mlflow.log_param("n_samples", len(X_train))

    mlflow.log_metric("train_rmse", train_rmse_lr)
    mlflow.log_metric("test_rmse", test_rmse_lr)
    mlflow.log_metric("train_r2", train_r2_lr)
    mlflow.log_metric("test_r2", test_r2_lr)
    mlflow.log_metric("train_mae", train_mae_lr)
    mlflow.log_metric("test_mae", test_mae_lr)

    # Generate model signature
    signature = infer_signature(X_train_scaled, y_pred_train_lr)

    # Save and register model
    mlflow.sklearn.log_model(
        sk_model=lr_model,
        artifact_path="model",
        signature=signature,
        registered_model_name=model_name,
        input_example=X_train_scaled[:5]
    )

    lr_run_id = run.info.run_id

    print("Linear Regression Model Performance:")
    print(f"ℹ️ Training R²: {train_r2_lr:.4f}")
    print(f"ℹ️ Test R²: {test_r2_lr:.4f}")
    print(f"ℹ️ Training RMSE: {train_rmse_lr:.4f}")
    print(f"ℹ️ Test RMSE: {test_rmse_lr:.4f}")
    print(f"ℹ️ Training MAE: {train_mae_lr:.4f}")
    print(f"ℹ️ Test MAE: {test_mae_lr:.4f}")

In [0]:
# ---------------------------------------------------
# Cell : Random Forest Model Training
# ---------------------------------------------------

# Start MLflow run - Random Forest
with mlflow.start_run(run_name="Random_Forest") as run:

    # Train model
    rf_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train_scaled, y_train)

    # Predictions
    y_pred_train_rf = rf_model.predict(X_train_scaled)
    y_pred_test_rf = rf_model.predict(X_test_scaled)

    # Performance evaluation
    train_rmse_rf = np.sqrt(mean_squared_error(y_train, y_pred_train_rf))
    test_rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_test_rf))
    train_r2_rf = r2_score(y_train, y_pred_train_rf)
    test_r2_rf = r2_score(y_test, y_pred_test_rf)
    train_mae_rf = mean_absolute_error(y_train, y_pred_train_rf)
    test_mae_rf = mean_absolute_error(y_test, y_pred_test_rf)

    # Log hyperparameters
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 10)
    mlflow.log_param("min_samples_split", 5)
    mlflow.log_param("min_samples_leaf", 2)
    mlflow.log_param("n_features", len(feature_columns))
    mlflow.log_param("n_samples", len(X_train))

    # Log metrics
    mlflow.log_metric("train_rmse", train_rmse_rf)
    mlflow.log_metric("test_rmse", test_rmse_rf)
    mlflow.log_metric("train_r2", train_r2_rf)
    mlflow.log_metric("test_r2", test_r2_rf)
    mlflow.log_metric("train_mae", train_mae_rf)
    mlflow.log_metric("test_mae", test_mae_rf)

    # Feature importance logging
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)

    # Save feature importance as artifact
    feature_importance.to_csv('/tmp/feature_importance.csv', index=False)
    mlflow.log_artifact('/tmp/feature_importance.csv')

    # Generate model signature
    signature = infer_signature(X_train_scaled, y_pred_train_rf)

    # Save and register model
    mlflow.sklearn.log_model(
        sk_model=rf_model,
        artifact_path="model",
        signature=signature,
        registered_model_name=model_name,
        input_example=X_train_scaled[:5]
    )

    rf_run_id = run.info.run_id

    print("Random Forest Model Performance:")
    print(f"ℹ️ Training R²: {train_r2_rf:.4f}")
    print(f"ℹ️ Test R²: {test_r2_rf:.4f}")
    print(f"ℹ️ Training RMSE: {train_rmse_rf:.4f}")
    print(f"ℹ️ Test RMSE: {test_rmse_rf:.4f}")
    print(f"ℹ️ Training MAE: {train_mae_rf:.4f}")
    print(f"ℹ️ Test MAE: {test_mae_rf:.4f}")
    print("\nTop 10 Important Features:")
    print(feature_importance.head(10))

In [0]:
# ---------------------------------------------------
# Cell : Model Performance Comparison and Visualization
# ---------------------------------------------------

# Create performance comparison dataframe
comparison_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest'],
    'Train_R2': [train_r2_lr, train_r2_rf],
    'Test_R2': [test_r2_lr, test_r2_rf],
    'Train_RMSE': [train_rmse_lr, train_rmse_rf],
    'Test_RMSE': [test_rmse_lr, test_rmse_rf],
    'Train_MAE': [train_mae_lr, train_mae_rf],
    'Test_MAE': [test_mae_lr, test_mae_rf],
    'Overfitting': [train_r2_lr - test_r2_lr, train_r2_rf - test_r2_rf]
})

print("ℹ️ Model Performance Comparison:")
print(comparison_df)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# R² score comparison
x_pos = np.arange(len(comparison_df['Model']))
width = 0.35
axes[0, 0].bar(x_pos - width/2, comparison_df['Train_R2'], width, label='Train R²', alpha=0.8)
axes[0, 0].bar(x_pos + width/2, comparison_df['Test_R2'], width, label='Test R²', alpha=0.8)
axes[0, 0].set_title('R² Score Comparison (Train vs Test)')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].set_xticks(x_pos)
axes[0, 0].set_xticklabels(comparison_df['Model'])
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# RMSE comparison
axes[0, 1].bar(x_pos - width/2, comparison_df['Train_RMSE'], width, label='Train RMSE', alpha=0.8)
axes[0, 1].bar(x_pos + width/2, comparison_df['Test_RMSE'], width, label='Test RMSE', alpha=0.8)
axes[0, 1].set_title('RMSE Comparison (Train vs Test)')
axes[0, 1].set_ylabel('RMSE')
axes[0, 1].set_xticks(x_pos)
axes[0, 1].set_xticklabels(comparison_df['Model'])
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Predicted vs Actual scatter plot (Linear Regression)
sample_size = min(1000, len(y_test))
sample_idx = np.random.choice(len(y_test), sample_size, replace=False)
axes[1, 0].scatter(y_test.iloc[sample_idx], y_pred_test_lr[sample_idx], alpha=0.6, s=10)
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 0].set_title(f'Linear Regression\nPredicted vs Actual (R²={test_r2_lr:.3f})')
axes[1, 0].set_xlabel('Actual')
axes[1, 0].set_ylabel('Predicted')
axes[1, 0].grid(True, alpha=0.3)

# Predicted vs Actual scatter plot (Random Forest)
axes[1, 1].scatter(y_test.iloc[sample_idx], y_pred_test_rf[sample_idx], alpha=0.6, s=10)
axes[1, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 1].set_title(f'Random Forest\nPredicted vs Actual (R²={test_r2_rf:.3f})')
axes[1, 1].set_xlabel('Actual')
axes[1, 1].set_ylabel('Predicted')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Overfitting analysis
print(f"\n📊 Overfitting Analysis:")
print(f"Linear Regression overfitting degree: {train_r2_lr - test_r2_lr:.4f}")
print(f"Random Forest overfitting degree: {train_r2_rf - test_r2_rf:.4f}")


In [0]:
# ---------------------------------------------------
# Cell : Best Model Selection and Validation
# ---------------------------------------------------

# Select best model based on test R²
if test_r2_lr > test_r2_rf:
    best_model = lr_model
    best_model_name = "Linear Regression"
    best_run_id = lr_run_id
    best_test_r2 = test_r2_lr
    best_predictions = y_pred_test_lr
else:
    best_model = rf_model
    best_model_name = "Random Forest"
    best_run_id = rf_run_id
    best_test_r2 = test_r2_rf
    best_predictions = y_pred_test_rf

print(f"ℹ️ Best Model: {best_model_name}")
print(f"ℹ️ Test R² Score: {best_test_r2:.4f}")
print(f"ℹ️ Prediction Accuracy: {best_test_r2*100:.1f}%")
print(f"ℹ️ Run ID: {best_run_id}")

# Prediction accuracy analysis by error range
prediction_errors = np.abs(y_test - best_predictions)
relative_errors = prediction_errors / (y_test + 1e-8)  # Prevent division by zero

print(f"\n📊 Prediction Error Analysis:")
print(f"Mean Absolute Error: {prediction_errors.mean():.2f}")
print(f"Mean Relative Error: {relative_errors.mean()*100:.1f}%")
print(f"Predictions within 10% error: {(relative_errors < 0.1).mean()*100:.1f}%")
print(f"Predictions within 20% error: {(relative_errors < 0.2).mean()*100:.1f}%")
print(f"Predictions within 30% error: {(relative_errors < 0.3).mean()*100:.1f}%")

In [0]:
# ---------------------------------------------------
# Cell : Model Version Management and Alias Setting
# ---------------------------------------------------

# Create MLflow client
client = mlflow.MlflowClient()

try:
    # Check all versions of registered model
    model_versions = client.search_model_versions(f"name='{model_name}'")

    print(f"Model '{model_name}' version information:")
    for version in model_versions:
        print(f"ℹ️ Version {version.version}:")
        print(f"   - Run ID: {version.run_id}")
        print(f"   - Status: {version.status}")
        
    # Find best model version
    best_version = None
    challenger_version = None
    
    for mv in model_versions:
        if mv.run_id == best_run_id:
            best_version = mv.version
        elif mv.run_id != best_run_id:
            challenger_version = mv.version

    if best_version:
        # Set 'champion' alias
        client.set_registered_model_alias(
            name=model_name,
            alias="champion",
            version=best_version
        )

        print(f"✅ 'champion' alias set for model {model_name} version {best_version}")

        # Set 'challenger' alias
        if challenger_version:
            client.set_registered_model_alias(
                name=model_name,
                alias="challenger",
                version=challenger_version
            )
            print(f"✅ 'challenger' alias set for model {model_name} version {challenger_version}")

        # Update model description
        client.update_model_version(
            name=model_name,
            version=best_version,
            description=f"""
            Coffee Order Quantity Demand Forecast Model ({best_model_name})
            - Model Type: {best_model_name}
            - Test R²: {best_test_r2:.4f}
            - Prediction Accuracy: {best_test_r2*100:.1f}%
            - Training Data: {X_train.shape[0]:,} records
            - Number of Features: {X_train.shape[1]}
            - Created: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            - Status: Champion Model
            """
        )

        print("✅ Model description updated successfully")

except Exception as e:
    print(f"❌ Error occurred while setting model alias: {e}")

In [0]:
# ---------------------------------------------------
# Cell : Load Best Model and Create Prediction Function
# ---------------------------------------------------

# Load the champion model
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}@champion")

print(f"✅ Champion model loaded successfully: {model_name}@champion")

# Test prediction with sample data
sample_data = X_test_scaled[:5]
sample_predictions = loaded_model.predict(sample_data)

print(f"\n📊 Sample Predictions:")
for i, pred in enumerate(sample_predictions):
    actual = y_test.iloc[i]
    print(f"Sample {i+1}: Predicted={pred:.2f}, Actual={actual:.2f}, Error={abs(pred-actual):.2f}")


## Simulation using champion model

In [0]:
# ---------------------------------------------------
# Cell : Order Quantity Prediction Simulation
# ---------------------------------------------------

# Order quantity prediction simulation for specific stores and products
sample_stores = ['P0001', 'P0100', 'P0500', 'P1000', 'P1500']
sample_products = ['C00001', 'C00005', 'C00010', 'C00015', 'C00020']

print("Order Quantity Prediction Simulation")
print("="*80)

simulation_results = []

for store in sample_stores:
    for product in sample_products:
        # Get store-product data
        store_product_data = pdf_ml[(pdf_ml['WERKS'] == store) & 
                                   (pdf_ml['MATNR'] == product)]
        
        if not store_product_data.empty:
            # Prepare features
            features = store_product_data[feature_columns].values
            features_scaled = scaler.transform(features)
            
            # Prediction
            predicted_qty = loaded_model.predict(features_scaled)[0]
            
            # Current information
            current_stock = store_product_data['current_stock'].values[0]
            avg_sales = store_product_data['avg_sales_qty'].values[0]
            safety_stock = store_product_data['safety_stock'].values[0]
            product_name = store_product_data['product_name'].values[0]
            city = store_product_data['city'].values[0]
            
            # Order recommendation
            recommended_order = max(0, predicted_qty)
            urgency = "Urgent" if current_stock < safety_stock else "Normal"
            
            simulation_results.append({
                'store': store,
                'product': product,
                'product_name': product_name,
                'city': city,
                'current_stock': current_stock,
                'safety_stock': safety_stock,
                'avg_sales': avg_sales,
                'predicted_qty': predicted_qty,
                'recommended_order': recommended_order,
                'urgency': urgency
            })
            
            print(f"\n☕️ {city} {store} - {product} ({product_name})")
            print(f"  - Current Stock: {current_stock:.0f} KG | Safety Stock: {safety_stock:.0f} KG")
            print(f"  - Average Sales: {avg_sales:.1f} KG/order")
            print(f"  - Predicted Order Qty: {predicted_qty:.1f} KG")
            print(f"  - Recommended Order: {recommended_order:.0f} KG ({urgency})")

# Simulation results summary
sim_df = pd.DataFrame(simulation_results)
urgent_orders = sim_df[sim_df['urgency'] == 'Urgent']

print("="*80)
print(f"\n📊 Order Simulation Summary:")
print(f"  - Total Analysis Target: {len(sim_df)} store-product combinations")
print(f"  - Urgent Orders Required: {len(urgent_orders)} cases")
print(f"  - Average Recommended Order Qty: {sim_df['recommended_order'].mean():.1f} KG")
print(f"  - Total Recommended Order Qty: {sim_df['recommended_order'].sum():.0f} KG")

if len(urgent_orders) > 0:
    print(f"\n🚨 Stores Requiring Urgent Orders:")
    for _, row in urgent_orders.iterrows():
        print(f"  - {row['city']} {row['store']}: {row['product_name']} ({row['recommended_order']:.0f} KG)")
